In [1]:
import pathlib
import pickle
import numpy as np
import scipy
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.svm import SVR, LinearSVR
from tqdm import tqdm

import importlib

In [2]:
import ensembles

importlib.reload(ensembles)

<module 'ensembles' from 'C:\\Users\\Vladimir\\PycharmProjects\\ML_Ensembles\\experiments\\ensembles\\__init__.py'>

In [3]:
data = pd.read_csv('kc_house_data.csv')

data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
y = data.price.to_numpy()
X = data.drop(columns=['price'])
X['date'] = pd.to_datetime(X.date.apply(lambda s: f"{s[0:4]}-{s[4:6]}-{s[6:8]}")).apply(lambda dt: dt.value)
X.drop(columns='id', inplace=True)

X.head()

,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1413158400000000000,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1418083200000000000,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,1424822400000000000,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,1418083200000000000,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1424217600000000000,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [5]:
X = X.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

print(f"{X_train.shape=}")
print(f"{X_test.shape=}")

X_train.shape=(17290, 19)
X_test.shape=(4323, 19)


In [6]:
X_train

array([[ 1.4011488e+18,  3.0000000e+00,  1.7500000e+00, ...,
        -1.2215200e+02,  2.7500000e+03,  1.3095000e+04],
       [ 1.4259456e+18,  2.0000000e+00,  1.0000000e+00, ...,
        -1.2229000e+02,  1.2700000e+03,  5.0000000e+03],
       [ 1.4249088e+18,  3.0000000e+00,  1.0000000e+00, ...,
        -1.2233500e+02,  1.1700000e+03,  7.8000000e+03],
       ...,
       [ 1.4120352e+18,  3.0000000e+00,  2.5000000e+00, ...,
        -1.2203200e+02,  1.6900000e+03,  2.6500000e+03],
       [ 1.4032224e+18,  1.0000000e+00,  7.5000000e-01, ...,
        -1.2232300e+02,  1.1700000e+03,  1.5000000e+04],
       [ 1.4272416e+18,  4.0000000e+00,  2.5000000e+00, ...,
        -1.2209900e+02,  3.0200000e+03,  5.9970000e+03]])

In [21]:
rf = ensembles.RandomForestMSE(
    n_estimators=250,
    feature_subsample_size=6,
    random_state=42,
    max_depth=None,
)

rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

print(f"RMSE: {mean_squared_error(y_true=y_test, y_pred=rf_pred) ** 0.5:.0f}")
print(f"r^2: {r2_score(y_true=y_test, y_pred=rf_pred):.4f}")

RMSE: 156860
r^2: 0.8372


In [17]:
boosting = ensembles.GradientBoostingMSE(
    n_estimators=250,
    feature_subsample_size=6,
    random_state=42,
    max_depth=7,
    learning_rate=0.05
)

boosting.fit(X_train, y_train)
boosting_predict = boosting.predict(X_test)

print(f"RMSE: {mean_squared_error(y_true=y_test, y_pred=boosting_predict) ** 0.5:.0f}")
print(f"r^2: {r2_score(y_true=y_test, y_pred=boosting_predict):.4f}")

MSE: 137418
r^2: 0.8751


In [19]:
ridge = Pipeline([
    ('scale', StandardScaler()),
    ('regression', Ridge(alpha=0.01))
])

ridge.fit(X_train, y_train)
ridge_predict = ridge.predict(X_test)

print(f"RMSE: {mean_squared_error(y_true=y_test, y_pred=ridge_predict) ** 0.5:.0f}")
print(f"r^2: {r2_score(y_true=y_test, y_pred=ridge_predict):.4f}")

MSE: 212120
r^2: 0.7024


In [20]:
svm = Pipeline([
    ('scale', StandardScaler()),
    ('regression', SVR(kernel='linear', C=1e4))
])

svm.fit(X_train, y_train)
svm_predict = svm.predict(X_test)

print(f"RMSE: {mean_squared_error(y_true=y_test, y_pred=svm_predict) ** 0.5:.0f}")
print(f"r^2: {r2_score(y_true=y_test, y_pred=svm_predict):.4f}")

MSE: 230030
r^2: 0.6500


Реализация метода для оценки разложения среднего эмпирического риска на шум, смещение и разброс

In [11]:
def compute_biase_variance(regressor, X, y, num_runs=1000):
    """
    :param regressor: sklearn estimator with fit(...) and predict(...) method
    :param X: numpy-array representing training set ob objects, shape [n_obj, n_feat]
    :param y: numpy-array representing target for training objects, shape [n_obj]
    :param num_runs: int, number of samples (s in the description of the algorithm)

    :returns: bias (float), variance (float), error (float)
    each value is computed using bootstrap
    """
    
    T = np.full((num_runs, X.shape[0]), np.nan, dtype=float)
    
    for run in tqdm(range(num_runs)):
        tr_idx = np.random.choice(X.shape[0], size=X.shape[0], replace=True)
        tst_idx = ~np.any((tr_idx[:, None] == np.arange(X.shape[0])[None, :]), axis=0)
        # print(tst_idx, f"{np.sum(tst_idx)/X.shape[0]:.2f}")
        
        X_tr = X[tr_idx]
        y_tr = y[tr_idx]
        
        X_tst = X[tst_idx]
        # y_tst = y[tst_idx]
        
        regressor.fit(X_tr, y_tr)
        
        y_pred = regressor.predict(X_tst)
        T[run, tst_idx] = y_pred
    
    # print(T)
    
    error = np.nanmean((T - y[None, :]) ** 2)
    bias = np.nanmean((np.nanmean(T, axis=0) - y) ** 2)
    variance = np.nanmean(
        np.nanmean(
            (T - np.nanmean(T, axis=0)[None, :]) ** 2,
            axis=0
        )
    )
    
    return bias, variance, error

Оценка BV-decomposition для моделей

In [12]:
rf_bvd = compute_biase_variance(rf, X, y, num_runs=100)
boosting_bvd = compute_biase_variance(boosting, X, y, num_runs=100)
ridge_bvd = compute_biase_variance(ridge, X, y, num_runs=100)
svm_bvd = compute_biase_variance(svm, X, y, num_runs=100)

100%|██████████| 100/100 [23:35<00:00, 14.15s/it]


In [18]:
path = pathlib.Path('./dumps/compare_bvd.pkl')

with open(path, "wb") as f:
    pickle.dump((rf_bvd, boosting_bvd, ridge_bvd, svm_bvd), file=f)

In [13]:
max_bias = max(rf_bvd[0], boosting_bvd[0], ridge_bvd[0], svm_bvd[0])
max_variance = max(rf_bvd[1], boosting_bvd[1], ridge_bvd[1], svm_bvd[1])
max_error = max(rf_bvd[2], boosting_bvd[2], ridge_bvd[2], svm_bvd[2])

In [15]:
for method_nm, method_rs in zip(['Random forest', 'Gradient boosting', 'Ridge', 'SVM'], [rf_bvd, boosting_bvd, ridge_bvd, svm_bvd]):
  bias, variance, error = method_rs
  print(f"{method_nm}:")
  print(f"\tbias = {bias / max_bias:.4f}\t\tvariance = {variance / max_variance:.4f}\t\terror = {error / max_error:.4f}")
  print()

Random forest:
	bias = 0.4201		variance = 0.6041		error = 0.4444

Gradient boosting:
	bias = 0.3071		variance = 1.0000		error = 0.3360

Ridge:
	bias = 0.8744		variance = 0.0924		error = 0.8783

SVM:
	bias = 1.0000		variance = 0.0540		error = 1.0000
